<a href="https://colab.research.google.com/github/shafakhadijah/DeepLearningTask/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Shafa Khadijah Rahmat

NPM: 2306319496

In [8]:
!pip install transformers

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification


In [2]:
# Initialize the pipeline
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Predict sentiment on a test sentence
sentence = "The company reported a significant increase in profits for the third quarter."
result = pipe(sentence)
print(result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'positive', 'score': 0.9996966123580933}]


In [3]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Tokenize input
inputs = tokenizer(sentence, return_tensors="pt")
outputs = model(**inputs)

# Get predicted class
predicted_class = outputs.logits.argmax(dim=-1).item()
print(f"Predicted class: {predicted_class}")


Predicted class: 2


In [4]:
# Synthetic dataset
synthetic_data = [
    ("The company reported record profits this year.", "positive"),
    ("The company's stock price plummeted due to a scandal.", "negative"),
    ("The market outlook remains uncertain amidst inflation fears.", "neutral"),
    ("Investors are optimistic about the future growth prospects.", "positive"),
]

# Check accuracy
correct = 0
total = len(synthetic_data)
for sentence, label in synthetic_data:
    result = pipe(sentence)
    predicted_label = result[0]['label'].lower()
    if predicted_label == label:
        correct += 1

accuracy = correct / total
print(f"Model accuracy on synthetic dataset: {accuracy * 100:.2f}%")


Model accuracy on synthetic dataset: 75.00%


In [5]:
import torch
import torch.nn as nn

class SimpleAttention(nn.Module):
    def __init__(self, model):
        super(SimpleAttention, self).__init__()
        self.model = model
        self.attention = nn.MultiheadAttention(embed_dim=model.config.hidden_size, num_heads=8)

    def forward(self, inputs):
        # Pass the input through the transformer model
        outputs = self.model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # Get the last hidden state
        attention_output, _ = self.attention(hidden_states, hidden_states, hidden_states)
        return attention_output

# Wrap the original model with attention
attention_model = SimpleAttention(model)


In [6]:
# Evaluate the model with attention
correct = 0
total = len(synthetic_data)

for sentence, label in synthetic_data:
    inputs = tokenizer(sentence, return_tensors="pt")
    attention_output = attention_model(inputs)

    # Assuming we use the first token's output for classification
    logits = attention_output[0, 0, :]
    predicted_class = logits.argmax(dim=-1).item()

    predicted_label = "positive" if predicted_class == 0 else "negative" if predicted_class == 1 else "neutral"

    if predicted_label == label:
        correct += 1

accuracy_with_attention = correct / total
print(f"Model accuracy with attention on synthetic dataset: {accuracy_with_attention * 100:.2f}%")


Model accuracy with attention on synthetic dataset: 25.00%


In [7]:
print(f"Accuracy without attention: {accuracy * 100:.2f}%")
print(f"Accuracy with attention: {accuracy_with_attention * 100:.2f}%")


Accuracy without attention: 75.00%
Accuracy with attention: 25.00%
